In [7]:
!ls ../../models/

bertweet-hate-speech	     robertuito-irony
robertuito-emotion-analysis  robertuito-lince-ner
robertuito-hate-speech	     robertuito-sentiment-analysis


In [63]:
%load_ext autoreload
%autoreload 2
from transformers import AutoModelForTokenClassification, AutoTokenizer
from pysentimiento.lince.ner import load_datasets

model = AutoModelForTokenClassification.from_pretrained("../../models/robertuito-lince-ner")

tokenizer = AutoTokenizer.from_pretrained("../../models/robertuito-lince-ner")
tokenizer.model_max_length = 128

train_dataset, dev_dataset, test_dataset = load_datasets(lang="es")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Reusing dataset lince (/home/jmperez/.cache/huggingface/datasets/lince/ner_spaeng/1.0.0/10d41747f55f0849fa84ac579ea1acfa7df49aa2015b60426bc459c111b3d589)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at /home/jmperez/.cache/huggingface/datasets/lince/ner_spaeng/1.0.0/10d41747f55f0849fa84ac579ea1acfa7df49aa2015b60426bc459c111b3d589/cache-f749fd5ca349b1d8.arrow
Loading cached processed dataset at /home/jmperez/.cache/huggingface/datasets/lince/ner_spaeng/1.0.0/10d41747f55f0849fa84ac579ea1acfa7df49aa2015b60426bc459c111b3d589/cache-875716555d36ff1f.arrow
Loading cached processed dataset at /home/jmperez/.cache/huggingface/datasets/lince/ner_spaeng/1.0.0/10d41747f55f0849fa84ac579ea1acfa7df49aa2015b60426bc459c111b3d589/cache-1afcf30fe2e4ebdb.arrow
Loading cached processed dataset at /home/jmperez/.cache/huggingface/datasets/lince/ner_spaeng/1.0.0/10d41747f55f0849fa84ac579ea1acfa7df49aa2015b60426bc459c111b3d589/cache-3208b28ae84d18ec.arrow
Loading cached processed dataset at /home/jmperez/.cache/huggingface/datasets/lince/ner_spaeng/1.0.0/10d41747f55f0849fa84ac579ea1acfa7df49aa2015b60426bc459c111b3d589/cache-77cc7027f242a489.arrow
Loading cached processed 

In [64]:
from pysentimiento.lince.ner import load_conll

test_path = "../../data/lince/ner_spaeng/test.conll"

test_data = load_conll(test_path)

len(test_data)

23527

In [91]:
from datasets import Dataset
from pysentimiento.lince.ner import preprocess_token, tokenize_and_align_labels

original_words = [[x[0] for x in sentence] for sentence in test_data]
original_langs = [[x[1] for x in sentence] for sentence in test_data]
ner = [[None] * len(x) for x in test_data]

for w, l in zip(original_words, ner):
    assert len(w) == len(l)

test_dataset = Dataset.from_dict(
    {"words": original_words, "lang": original_langs, "labels": ner}
)


test_dataset = test_dataset.map(
    lambda x: {
        "words": [preprocess_token(word, "es") for word in x["words"]]
    }
)


  0%|          | 0/23527 [00:00<?, ?ex/s]

In [92]:

tokenize_fun = lambda x: tokenize_and_align_labels(x, tokenizer)
test_dataset = test_dataset.map(
    tokenize_fun, batched=True, batch_size=32, 
)

test_dataset = test_dataset.remove_columns(["labels"])

  0%|          | 0/736 [00:00<?, ?ba/s]

In [93]:
from tqdm.auto import tqdm

problematic_instances = []

for idx, row in tqdm(enumerate(test_dataset), total=len(test_dataset)):
    word_ids = row["word_ids"]
    input_ids = row["input_ids"]
    words = original_words[idx]
    if not (word_ids[-2] + 1) == len(words):
        problematic_instances.append(idx)

print(f"{len(problematic_instances)} problematic instances")

  0%|          | 0/23527 [00:00<?, ?it/s]

2 problematic instances


In [95]:
idx = problematic_instances[0]

row = test_dataset[idx]

words = original_words[idx]

print(words)

[tokenizer.decode(t) for t in row["input_ids"]]

['⚪️⚪️⚪️⚪️⚪️⚪️⚪️', '⚪️⚪️⚪️⚪️⚪️⚪️⚪️', '⚪️⚪️⚪️⚪️⚪️⚪️⚪️', '⚪️⚪️⚪️⚪️⚪️⚪️⚪️', '⚪️⚪️⚪️⚪️⚪️⚪️⚪️', '⚪️⚪️⚪️⚪️⚪️⚪️🔴']


['<s>',
 'emoji',
 'circulo',
 'blanco',
 '',
 'emoji',
 '',
 '',
 'emoji',
 'circulo',
 'blanco',
 '',
 'emoji',
 '',
 '',
 'emoji',
 'circulo',
 'blanco',
 '',
 'emoji',
 '',
 '',
 'emoji',
 'circulo',
 'blanco',
 '',
 'emoji',
 '',
 '',
 'emoji',
 'circulo',
 'blanco',
 '',
 'emoji',
 '',
 '',
 'emoji',
 'circulo',
 'blanco',
 '',
 'emoji',
 '',
 '',
 'emoji',
 'circulo',
 'blanco',
 '',
 'emoji',
 'emoji',
 'circulo',
 'blanco',
 '',
 'emoji',
 '',
 '',
 'emoji',
 'circulo',
 'blanco',
 '',
 'emoji',
 '',
 '',
 'emoji',
 'circulo',
 'blanco',
 '',
 'emoji',
 '',
 '',
 'emoji',
 'circulo',
 'blanco',
 '',
 'emoji',
 '',
 '',
 'emoji',
 'circulo',
 'blanco',
 '',
 'emoji',
 '',
 '',
 'emoji',
 'circulo',
 'blanco',
 '',
 'emoji',
 '',
 '',
 'emoji',
 'circulo',
 'blanco',
 '',
 'emoji',
 'emoji',
 'circulo',
 'blanco',
 '',
 'emoji',
 '',
 '',
 'emoji',
 'circulo',
 'blanco',
 '',
 'emoji',
 '',
 '',
 'emoji',
 'circulo',
 'blanco',
 '',
 'emoji',
 '',
 '',
 'emoji',
 'circulo',
 'bl

In [22]:
from transformers import DataCollatorForTokenClassification, Trainer, TrainingArguments
data_collator = DataCollatorForTokenClassification(tokenizer)

train_args = TrainingArguments(
    output_dir="./test/",
    per_device_eval_batch_size=32,
    do_train=False,
)

trainer_args = {
    "model": model,
    "args": train_args,
    "eval_dataset": test_dataset,
    "data_collator": data_collator,
    "tokenizer": tokenizer,
}

eval_trainer = Trainer(**trainer_args)

In [23]:
ret = eval_trainer.predict(test_dataset)

The following columns in the test set  don't have a corresponding argument in `RobertaForTokenClassification.forward` and have been ignored: word_ids, words, lang.
***** Running Prediction *****
  Num examples = 23527
  Batch size = 32


In [30]:
word_ids[-2], len(words)

(10, 12)

In [24]:
from tqdm.auto import tqdm
from pysentimiento.lince.ner import id2label, label2id

outputs = []

for idx, row in tqdm(enumerate(test_dataset), total=len(test_dataset)):
    word_ids = row["word_ids"]
    input_ids = row["input_ids"]
    words = original_words[idx]
    assert (word_ids[-2] + 1) == len(words)
    preds = ret.predictions[idx]

    sentence_output = []
    current_word_id = None

    for word_id, token_id, pred in zip(word_ids, input_ids, preds):
        token = tokenizer.decode(token_id)
        if current_word_id != word_id and word_id is not None:
            current_word_id = word_id
            label = id2label[pred.argmax()]
            sentence_output.append([words[current_word_id], label])
    outputs.append(sentence_output)


  0%|          | 0/23527 [00:00<?, ?it/s]

AssertionError: 

In [25]:
words, row["words"]

(['makeupbymariajose',
  "'s",
  'photo',
  'https://t.co/LvRc5JJxvJ',
  'denle',
  'follow',
  'para',
  'citas',
  'de',
  'maquillaje',
  '☺',
  '️'],
 ['makeupbymariajose',
  "'s",
  'photo',
  'https://url',
  'denle',
  'follow',
  'para',
  'citas',
  'de',
  'maquillaje',
  '☺',
  '️'])

In [32]:
def write_conll(path, data):
    with open(path, "w") as f:
        for sentence in data:
            for row in sentence:
                line = "\t".join(row)
                f.write(f"{line}\n")
            f.write("\n")

write_conll("test.conll", outputs)